In [1]:
import duckdb

I'm going to work with the data of yellow taxis in 2025.

In [2]:
con = duckdb.connect("taxis.db")

In [3]:
con.execute("""
CREATE OR REPLACE TABLE taxis AS
SELECT *
FROM 'data/taxis/yellow_tripdata_2025-*.parquet'
""")

In [5]:
con.execute("SHOW TABLES").fetchall()

[('taxis',)]

In [7]:
con.execute("""
SELECT COUNT(*)
FROM taxis
""").fetchall()

[(27982347,)]

In [10]:
con.execute("""
SELECT *
FROM taxis
LIMIT 5
""").fetch_df().T

,0,1,2,3,4
VendorID,1,1,1,2,2
tpep_pickup_datetime,2025-01-01 00:18:38,2025-01-01 00:32:40,2025-01-01 00:44:04,2025-01-01 00:14:27,2025-01-01 00:21:34
tpep_dropoff_datetime,2025-01-01 00:26:59,2025-01-01 00:35:13,2025-01-01 00:46:01,2025-01-01 00:20:01,2025-01-01 00:25:06
passenger_count,1,1,1,3,3
trip_distance,1.6,0.5,0.6,0.52,0.66
RatecodeID,1,1,1,1,1
store_and_fwd_flag,N,N,N,N,N
PULocationID,229,236,141,244,244
DOLocationID,237,237,141,244,116
payment_type,1,1,1,2,2


Before I run the query, I want to explain the choices I made and why. My goal is to calculate how much taxis are really charging per unit of service, either per mile or per minute. To do this, I first need to define what I mean by “effective fare.” I decided to build this value using the parts of the bill that always belong to the taxi itself: the base fare, the extra charges like night or rush hour, the MTA tax, the improvement surcharge, the congestion surcharge, and also the airport fee and the new CBD congestion fee. I use the function COALESCE to make sure that if one of these fields is missing or null, it becomes zero instead of creating errors. I do not include tolls or tips, because tolls are passed through to other companies and tips are inconsistent (for example, tips in cash are not even reported).

I also calculate the trip duration in seconds, using the pickup and drop-off times, and then I convert it into minutes. With this, I can divide the effective fare by the trip distance to get a price per mile, and I can also divide the effective fare by the duration in minutes to get a price per minute.

For quality control, I filter out bad or strange trips. I remove trips with zero or negative distance, and I only keep trips longer than 60 seconds. I also remove trips with very large distances, more than 100 miles, because these are probably errors. I only keep trips where the fare and the total amount are not negative. Finally, I only use trips with valid payment types (card, cash, or Flex Fare), and I exclude things like disputes or voided charges, because they don’t represent real payments.

This way, the data is cleaner, and the calculations for price per mile and price per minute are more reliable. Now I can create a new table with these metrics that I will use in the analysis.

In [11]:
con.execute(
"""
CREATE OR REPLACE TABLE taxis_metrics AS
WITH base AS (
  SELECT
    *,
    DATEDIFF('second', tpep_pickup_datetime, tpep_dropoff_datetime)         AS trip_seconds,
    fare_amount
    + COALESCE(extra,0)
    + COALESCE(mta_tax,0)
    + COALESCE(improvement_surcharge,0)
    + COALESCE(congestion_surcharge,0)
    + COALESCE(airport_fee,0)
    + COALESCE(cbd_congestion_fee,0)                                        AS effective_fare
  FROM taxis
)
SELECT
  *,
  trip_seconds/60.0                                                         AS dur_min,
  CASE WHEN trip_distance > 0 THEN effective_fare / trip_distance END       AS ep_per_mile,
  CASE WHEN trip_seconds > 0 THEN effective_fare / (trip_seconds/60.0) END  AS ep_per_min
FROM base
WHERE
  trip_distance > 0
  AND trip_seconds > 60
  AND trip_distance <= 100
  AND fare_amount >= 0
  AND total_amount >= 0
  AND payment_type IN (0,1,2)
"""
)

In [12]:
con.execute("""
SELECT
    tpep_pickup_datetime,
    tpep_dropoff_datetime,
    trip_distance,
    dur_min,
    effective_fare,
    ep_per_mile,
    ep_per_min,
    payment_type,
    RatecodeID
FROM taxis_metrics
LIMIT 10
""").fetch_df()


,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,dur_min,effective_fare,ep_per_mile,ep_per_min,payment_type,RatecodeID
0,2025-01-01 00:18:38,2025-01-01 00:26:59,1.60,8.350000,17.5,10.937500,2.095808,1,1
1,2025-01-01 00:32:40,2025-01-01 00:35:13,0.50,2.550000,12.6,25.200000,4.941176,1,1
2,2025-01-01 00:44:04,2025-01-01 00:46:01,0.60,1.950000,12.6,21.000000,6.461538,1,1
3,2025-01-01 00:14:27,2025-01-01 00:20:01,0.52,5.566667,9.7,18.653846,1.742515,2,1
4,2025-01-01 00:21:34,2025-01-01 00:25:06,0.66,3.533333,8.3,12.575758,2.349057,2,1
5,2025-01-01 00:48:24,2025-01-01 01:08:26,2.63,20.033333,24.1,9.163498,1.202995,2,1
6,2025-01-01 00:14:47,2025-01-01 00:16:15,0.40,1.466667,11.9,29.750000,8.113636,1,1
7,2025-01-01 00:39:27,2025-01-01 00:51:51,1.60,12.400000,19.6,12.250000,1.580645,1,1
8,2025-01-01 00:53:43,2025-01-01 01:13:23,2.80,19.666667,26.6,9.500000,1.352542,1,1
9,2025-01-01 00:00:02,2025-01-01 00:09:36,1.71,9.566667,16.4,9.590643,1.714286,2,1


When I look at the columns ep_per_mile and ep_per_min, I see them as two different ways of understanding how much money the taxi is really charging for the service. The first one, ep_per_mile, tells me how many dollars the passenger paid for each mile of the trip. This number is usually higher for short trips, because the fixed surcharges make the cost per mile larger when the distance is small. The second one, ep_per_min, shows how many dollars are paid for each minute of the trip. This helps me see the time cost, and it can go up if the trip is short but has extra fees, or if there is heavy traffic and the trip takes longer. By comparing these two metrics, I can understand better if the pricing is more influenced by distance, by time, or by the fixed charges, and I can also compare how this changes across the different Rate Codes, as I am going to do in the next cell:

In [13]:
con.execute(
"""
WITH map AS (
  SELECT 1 AS RatecodeID, 'Standard' AS rate_label UNION ALL
  SELECT 2,'JFK' UNION ALL
  SELECT 3,'Newark' UNION ALL
  SELECT 4,'Nassau/Westchester' UNION ALL
  SELECT 5,'Negotiated fare' UNION ALL
  SELECT 6,'Group ride' UNION ALL
  SELECT 99,'Null/unknown'
)
SELECT
  m.rate_label,
  COUNT(*)                         AS trips,
  ROUND(median(ep_per_mile), 2)    AS med_ep_per_mile,
  ROUND(avg(ep_per_mile), 2)       AS avg_ep_per_mile,
  ROUND(median(ep_per_min), 2)     AS med_ep_per_min,
  ROUND(avg(ep_per_min), 2)        AS avg_ep_per_min,
  ROUND(quantile(ep_per_mile,0.25),2) AS q1_mile,
  ROUND(quantile(ep_per_mile,0.75),2) AS q3_mile
FROM taxis_metrics y
LEFT JOIN map m USING (RatecodeID)
WHERE ep_per_mile IS NOT NULL AND ep_per_min IS NOT NULL
GROUP BY 1
ORDER BY 1;

"""
).fetch_df()

,rate_label,trips,med_ep_per_mile,avg_ep_per_mile,med_ep_per_min,avg_ep_per_min,q1_mile,q3_mile
0,Group ride,4,7.89,14.30,0.81,1.29,1.41,13.07
1,JFK,678072,4.32,7.41,1.54,1.79,4.03,4.56
2,Nassau/Westchester,54706,5.66,5.84,2.64,2.78,5.08,6.37
3,Negotiated fare,104851,8.11,29.40,2.65,4.80,5.96,12.38
4,Newark,56799,5.37,9.92,2.54,2.74,5.05,6.50
5,Null/unknown,304713,4.44,6.79,0.77,0.85,3.55,6.67
6,Standard,18877787,11.24,12.94,1.67,1.87,8.32,15.34
7,None,4968072,7.39,44.84,1.35,1.47,5.55,9.88


When I compare the numbers across Rate Codes, I see that the values of cost per mile and cost per minute do not always tell the same story. For example, JFK trips look cheap per mile because the trip is long and has a fixed price, but if I look at cost per minute it is closer to the Standard trips. Newark trips, on the other hand, show a higher cost per minute, which probably reflects time lost in traffic. Negotiated fares are very unstable, with a lot of variation, and Group rides have too few observations to trust the results. Standard trips stay in the middle and represent most of the market, so they give me a solid reference point.

From this, I think that the cost per minute is the better metric for this problem, because it is less distorted by trip length or fixed-rate codes. Cost per mile can be misleading when there are long airport rides with flat prices, but cost per minute gives me a more consistent way to compare how much passengers are really paying for the time they spend in the taxi.

My conclusion is that the pricing system in New York taxis is not fully aligned with the real cost structure of the trips. Some Rate Codes, like JFK, give passengers a low cost per mile, while others like Newark push the cost per minute much higher. This means that the unit economics change a lot depending on the code, and that could be unfair or inefficient. A better pricing system would try to balance distance and time in a way that is more uniform across Rate Codes.

----------------------


I think my approach is effective because it focuses on what the taxi actually earns from each trip, not just the total bill. By creating the "effective fare" metric, I can separate the real taxi revenue from things like tips and tolls that don't belong to the driver. The comparison between cost per mile and cost per minute helps me understand the pricing better than just looking at one number. The main advantage is that this method works well for different types of trips - short ones, long ones, and airport rides. It also cleans the data properly by removing weird trips that would mess up the results. However, there are some limitations. The approach assumes that all the charges I include really go to the taxi, but some might be shared with other companies. Also, I'm not considering external factors like traffic patterns, weather, or demand that could affect pricing. The filtered data might miss some valid edge cases, and the analysis doesn't account for seasonal changes or special events that could change how people use taxis.

For the next iteration, I would not only measure how much taxis are charging per mile or per minute, but I would try to build an optimization model that combines both distance and time. The idea is to create a formula that sets the price of a trip in a way that better reflects the real costs of driving, while also helping drivers and companies maximize their earnings. For example, the model could use historical data to learn how distance and time contribute to the effective fare, and then test different weights to find the balance that gives the highest gain without making the trips too expensive for passengers. This way, instead of just looking at one unit metric at a time, I would propose a data-driven way to redesign the pricing system so it is fair, consistent, and profitable.